In [1]:
import os
import tempfile
from typing import Any, Optional
from uuid import UUID
from pydantic import BaseModel
from fastapi import UploadFile
from backend.axon.in_come import File
from backend.settings.backend_setting import PGDocStore

import re
from dotenv import load_dotenv, find_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from file_utils import sanitize_filename, open_file, save_file, compute_sha1_from_content, compute_sha1_from_file
# from gpt2brain import llm_core
import openai
from logger import setup_logger
from gpt2brain import OpenAIBrain

from langchain.document_loaders import PDFPlumberLoader, PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector
from langchain.vectorstores.pgembedding import PGEmbedding
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document


# Create a logger for this module
logger = setup_logger(__name__)

# Create a OpenAIBrain instance
brain = OpenAIBrain()



In [1]:
from backend.axon.in_come import File
from backend.settings.backend_setting import PGDocStore
from backend.neuron.neuron import Memory

from langchain.document_loaders import PDFPlumberLoader
file = File()    
file.import_path(filepath="/mnt/d/ai/cognimesh/output/katoAnticoagulationStrategiesExtracorporeal2021.pdf",
                 chunk_size=1000,
                 chunk_overlap=50
                 )
neuron = Memory()
embed = neuron.create_vector(file = file, 
                             loader_class=PDFPlumberLoader)

store = PGDocStore()
doc_id = store.add_document(file=file)
vector_ids = store.add_vectors(doc_id, embed=embed)
store.update_document_vectors(doc_id, vector_ids)
store.close_connection()

In [3]:
file.vectors_ids

UUID('79daea53-f994-b5d3-a2eb-7b4406e05275')

In [ ]:
load_dotenv(find_dotenv())
host= os.getenv('PG_HOST')
port= os.getenv('PG_PORT')
user= os.getenv('PG_USER')
password= os.getenv('PG_PASS')
dbname= 'cognimesh'

loader = PDFPlumberLoader("/mnt/d/ai/cognimesh/output/katoAnticoagulationStrategiesExtracorporeal2021.pdf")
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=4000, chunk_overlap=200)
pages = loader.load()
docs = text_splitter.split_documents(pages)
embeddings = OpenAIEmbeddings()
CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}?client_encoding=utf8"
COLLECTION_NAME = "documents"

store = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    # table_name="vectors",
    embedding_function=embeddings,
)

store.add_documents(docs)

# db = PGEmbedding.from_documents(
#     embedding=embeddings,
#     documents=docs,
#     collection_name=COLLECTION_NAME,
#     connection_string=CONNECTION_STRING,
#     pre_delete_collection=False,
# )
